# コード7-1：偏微分方程式の数値計算の実装

『ウイルス感染の数理モデルとシミュレーション ーデータを定量的に理解するー』

偏微分方程式の数値計算の実装の例をまとめる。

## ライブラリ、パッケージ（モジュール）の読み込み

In [2]:
## 必要があればインストール
# !pip3 install matplotlib numpy gillespy2 time

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gillespy2
import time ## 実行時間測定

## 偏微分方程式の数値計算

式(7.2)

$$
\frac{d T(t)}{dt} =
$$

等価な式(7.5)を実験データに合わせて変形した式(7.6)

$$
\frac{d T(t)}{dt} =
$$

$V(t) = V_I(t) + V_{NI}(t), V_\theta(t) = \alpha V_I(t), \beta_\theta = \frac{\beta}{\alpha}, f_\theta = \alpha f$であるので

式(7.2)を以下のように変形する。

$$
\frac{d T(t)}{dt} = 
$$

式(7.2)を変形した偏微分方程式と式(7.5)の常微分方程式をそれぞれ数値計算することで比較する。

https://www.frontiersin.org/articles/10.3389/fmicb.2018.00601/full

The model equations were discretized in space, i.e., age, and integrated in time using the method of lines (MOL) approach (Sadiku and Obiozor, 2000; Shakeri and Dehghan, 2008) where the partial derivatives in age were approximated by finite-differences and the solution at the grid points was integrated along lines in time. We integrated the equations using the Matlab® ordinary differential equation Runge-Kutta solver ode45.

In [3]:
## 偏微分方程式の計算

In [4]:
## 常微分方程式の計算